# Load Libraries

In [138]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from keras.models import model_from_json
import pandas as pd
from hdfs import InsecureClient

# Connect to Hadoop

In [139]:
client_hdfs = InsecureClient('http://awscdh6-ma.sap.local:9870')

# Load Model

Model Structure:

In [140]:
with client_hdfs.read('/user/hue/TBR/ISR/ISR_BARMER/ISR_DSP/model/model_structure.json', encoding='utf-8') as reader:
    loaded_model_json = reader.read()
    reader.close()

model = model_from_json(loaded_model_json)

In [141]:
# load json and create model

#json_file = open('model/model_structure.json','r')
#loaded_model_json = json_file.read()
#json_file.close()
#model = model_from_json(loaded_model_json)

Model Weights:

In [142]:
download_path = client_hdfs.download('/user/hue/TBR/ISR/ISR_BARMER/ISR_DSP/model/model_weights.h5', 'download', overwrite=True)

In [143]:
model.load_weights(download_path)

In [144]:
#model.load_weights("model/model_weights.h5")

Compile model:

In [145]:
# compile loaded model 
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Load Data

In [146]:
def load_dataset(prefix):
    # load data and labels
    X = load_dataset_group(prefix)  
   
    # return dataset
    return X

In [147]:
# load a dataset group
def load_dataset_group(group):
    
    # load all 9 files as a single array
    filenames = list()
    
    # total acceleration
    filenames += ['/Inertial Signals/total_acc_x.txt',
                  '/Inertial Signals/total_acc_y.txt',
                  '/Inertial Signals/total_acc_z.txt']
    
    # body acceleration
    filenames += ['/Inertial Signals/body_acc_x.txt',
                  '/Inertial Signals/body_acc_y.txt',
                  '/Inertial Signals/body_acc_z.txt']
    
    # body gyroscope
    filenames += ['/Inertial Signals/body_gyro_x.txt',
                  '/Inertial Signals/body_gyro_y.txt',
                  '/Inertial Signals/body_gyro_z.txt']
    
    # load input data
    X = load_group(filenames, group)
       
    # return X
    return X

In [148]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, group):
    loaded = list()
    
    for name in filenames:

        data = load_file(group+name)
        loaded.append(data)
    
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [149]:
# load a single file as a numpy array
def load_file(filepath):
    #dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    
    path = '/user/hue/TBR/ISR/ISR_BARMER/ISR_DSP/' + filepath
    
    with client_hdfs.read(path, encoding = 'utf-8') as reader:
        dataframe = pd.read_csv(reader, header=None, delim_whitespace=True)   
        
    return dataframe.values

In [150]:
# load data
X = load_dataset('data_predictions')

# Make Predictions

In [151]:
def make_prediction(X):
    # define parameters
    verbose = 0
    samples = X.shape[0]
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features   

    
    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    X = X.reshape((samples, time_steps, rows, columns, channels))
        
    # fit network
    class_predictions = model.predict_classes(X, verbose=verbose)    
    
    return class_predictions

In [152]:
predictions = pd.DataFrame(make_prediction(X))

In [153]:
predictions

,0
0,4
1,4
2,4
3,4
4,4
...,...
2942,1
2943,1
2944,1
2945,1


# Save Predictions

In [163]:
with client_hdfs.write('/tmp/tbr/DSP/ISR_TBR_DSP_DATA_TEST.csv', encoding = 'utf-8', overwrite=True) as writer:
    predictions.to_csv(writer)

In [161]:
#path = '/user/hue/TBR/ISR/ISR_BARMER/ISR_DSP/data_predictions/predictions.csv'
#    
#with client_hdfs.write('/user/hue/TBR/ISR/ISR_BARMER/ISR_DSP/data_predictions/predictions.csv', encoding = 'utf-8', overwrite=True) as writer:
#    predictions.to_csv(writer)   

In [162]:
# Creating a simple Pandas DataFrame
liste_hello = ['hello1','hello2']
liste_world = ['world1','world2']
df = pd.DataFrame(data = {'hello' : liste_hello, 'world': liste_world})
 
## Writing Dataframe to hdfs
#with client_hdfs.write('/user/hue/TBR/ISR/ISR_BARMER/ISR_DSP/data_predictions/predictions.csv', encoding = 'utf-8') as writer:
#    df.to_csv(writer)